In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

/Users/adi2/Documents/ragProject/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 2 PDF files to process

Processing: MOD1.pdf
  ✓ Loaded 45 pages

Processing: MOD2.pdf
  ✓ Loaded 52 pages

Total documents loaded: 97


In [3]:
all_pdf_documents

[Document(metadata={'producer': 'macOS Version 14.1.2 (Build 23B92) Quartz PDFContext, AppendMode 1.1', 'creator': 'AWS Training and Certification', 'creationdate': "D:20230831205722Z00'00'", 'author': 'AWS Training and Certification', 'title': 'AWS Academy Cloud Architecting', 'subject': 'Module 01 Student Guide', 'moddate': "D:20240130164223Z00'00'", 'source': '../data/pdf/MOD1.pdf', 'total_pages': 45, 'page': 0, 'page_label': '1', 'source_file': 'MOD1.pdf', 'file_type': 'pdf'}, page_content='AWS Academy Cloud Architecting \nModule 01 Student Guide \nVersion 2.0.13 \n200-ACACAD-20-EN-SG'),
 Document(metadata={'producer': 'macOS Version 14.1.2 (Build 23B92) Quartz PDFContext, AppendMode 1.1', 'creator': 'AWS Training and Certification', 'creationdate': "D:20230831205722Z00'00'", 'author': 'AWS Training and Certification', 'title': 'AWS Academy Cloud Architecting', 'subject': 'Module 01 Student Guide', 'moddate': "D:20240130164223Z00'00'", 'source': '../data/pdf/MOD1.pdf', 'total_pages

In [4]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [5]:
chunks=split_documents(all_pdf_documents)
chunks

Split 97 documents into 154 chunks

Example chunk:
Content: AWS Academy Cloud Architecting 
Module 01 Student Guide 
Version 2.0.13 
200-ACACAD-20-EN-SG...
Metadata: {'producer': 'macOS Version 14.1.2 (Build 23B92) Quartz PDFContext, AppendMode 1.1', 'creator': 'AWS Training and Certification', 'creationdate': "D:20230831205722Z00'00'", 'author': 'AWS Training and Certification', 'title': 'AWS Academy Cloud Architecting', 'subject': 'Module 01 Student Guide', 'moddate': "D:20240130164223Z00'00'", 'source': '../data/pdf/MOD1.pdf', 'total_pages': 45, 'page': 0, 'page_label': '1', 'source_file': 'MOD1.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'macOS Version 14.1.2 (Build 23B92) Quartz PDFContext, AppendMode 1.1', 'creator': 'AWS Training and Certification', 'creationdate': "D:20230831205722Z00'00'", 'author': 'AWS Training and Certification', 'title': 'AWS Academy Cloud Architecting', 'subject': 'Module 01 Student Guide', 'moddate': "D:20240130164223Z00'00'", 'source': '../data/pdf/MOD1.pdf', 'total_pages': 45, 'page': 0, 'page_label': '1', 'source_file': 'MOD1.pdf', 'file_type': 'pdf'}, page_content='AWS Academy Cloud Architecting \nModule 01 Student Guide \nVersion 2.0.13 \n200-ACACAD-20-EN-SG'),
 Document(metadata={'producer': 'macOS Version 14.1.2 (Build 23B92) Quartz PDFContext, AppendMode 1.1', 'creator': 'AWS Training and Certification', 'creationdate': "D:20230831205722Z00'00'", 'author': 'AWS Training and Certification', 'title': 'AWS Academy Cloud Architecting', 'subject': 'Module 01 Student Guide', 'moddate': "D:20240130164223Z00'00'", 'source': '../data/pdf/MOD1.pdf', 'total_pages

In [6]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity


In [7]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("HTTPSConnection(host=\'huggingface.co\', port=443): Failed to resolve \'huggingface.co\' ([Errno 8] nodename nor servname provided, or not known)"))'), '(Request ID: d0dd0928-5d0e-486f-b10a-3861a9ee1fa0)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json
Retrying in 1s [Retry 1/5].


Loading embedding model: all-MiniLM-L6-v2


'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("HTTPSConnection(host=\'huggingface.co\', port=443): Failed to resolve \'huggingface.co\' ([Errno 8] nodename nor servname provided, or not known)"))'), '(Request ID: e10d0d35-4e91-421a-99f3-e4a0bf9fd367)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json
Retrying in 2s [Retry 2/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("HTTPSConnection(host=\'huggingface.co\', port=443): Failed to resolve \'huggingface.co\' ([Errno 8] nodename nor servname provided, or not known)"))'), '(Request ID: 00450777-d0ab-4068-a3b9-e906394e45be)')' thrown while requesting HEAD https://

Model loaded successfully. Embedding dimension: 384


In [8]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore
    

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [9]:
chunks

[Document(metadata={'producer': 'macOS Version 14.1.2 (Build 23B92) Quartz PDFContext, AppendMode 1.1', 'creator': 'AWS Training and Certification', 'creationdate': "D:20230831205722Z00'00'", 'author': 'AWS Training and Certification', 'title': 'AWS Academy Cloud Architecting', 'subject': 'Module 01 Student Guide', 'moddate': "D:20240130164223Z00'00'", 'source': '../data/pdf/MOD1.pdf', 'total_pages': 45, 'page': 0, 'page_label': '1', 'source_file': 'MOD1.pdf', 'file_type': 'pdf'}, page_content='AWS Academy Cloud Architecting \nModule 01 Student Guide \nVersion 2.0.13 \n200-ACACAD-20-EN-SG'),
 Document(metadata={'producer': 'macOS Version 14.1.2 (Build 23B92) Quartz PDFContext, AppendMode 1.1', 'creator': 'AWS Training and Certification', 'creationdate': "D:20230831205722Z00'00'", 'author': 'AWS Training and Certification', 'title': 'AWS Academy Cloud Architecting', 'subject': 'Module 01 Student Guide', 'moddate': "D:20240130164223Z00'00'", 'source': '../data/pdf/MOD1.pdf', 'total_pages

In [10]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 154 texts...


Batches: 100%|██████████| 5/5 [00:06<00:00,  1.27s/it]


Generated embeddings with shape: (154, 384)
Adding 154 documents to vector store...
Successfully added 154 documents to vector store
Total documents in collection: 154


In [11]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [12]:
rag_retriever

In [13]:
rag_retriever.retrieve("What is caching?")

Retrieving documents for query: 'What is caching?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 26.53it/s]

Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


[{'id': 'doc_be3f3362_124',
  'content': 'Caching is a technique to make future requests faster and reduce network throughput by \ntemporarily storing data in an intermediary location between the requester and the permanent \nstorage.\nIn the anti-pattern example, no caching service is used. When anyone requests a file from one \nof the Amazon Simple Storage Service (Amazon S3) buckets, each request takes the same \namount of time to complete, and each request costs the same.\nIn the best-practice-pattern example, the infrastructure uses Amazon CloudFront in front of \nAmazon S3 to providing caching. In this scenario, the initial request checks for the file in \nAmazon CloudFront. If it is not found, CloudFront requests the file from Amazon S3. CloudFront \nthen stores a copy of the file at an edge location close to the user, and sends a copy to the user \nwho made the request. Subsequent requests for the file are retrieved from the (now closer) \nedge location in CloudFront instead of

In [14]:
rag_retriever.retrieve("What is AWS")

Retrieving documents for query: 'What is AWS'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 52.34it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_b776a304_116',
  'content': '(Amazon SQS), Amazon DynamoDB, Elastic Load Balancing, Amazon Simple Email Service \n(Amazon SES), and Amazon Cognito.\nAWS Training and Certification Module 2: Introducing Cloud Architecting \n© 2023, Amazon Web Services, Inc. or its affiliates. All rights reserved. 31',
  'metadata': {'creationdate': "D:20230410212416Z00'00'",
   'author': 'AWS Training and Certification',
   'producer': 'macOS Version 14.1.2 (Build 23B92) Quartz PDFContext, AppendMode 1.1',
   'content_length': 267,
   'moddate': "D:20240130171949Z00'00'",
   'subject': 'Module 02 Student Guide',
   'creator': 'AWS Training and Certification',
   'total_pages': 52,
   'source': '../data/pdf/MOD2.pdf',
   'file_type': 'pdf',
   'doc_index': 116,
   'source_file': 'MOD2.pdf',
   'page': 30,
   'title': 'AWS Academy Cloud Architecting',
   'page_label': '31'},
  'similarity_score': 0.2726931571960449,
  'distance': 0.7273068428039551,
  'rank': 1},
 {'id': 'doc_f1fa1b0a_9',
  '

In [17]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage

In [20]:
class GroqLLM:
    def __init__(self, model_name: str = "llama-3.3-70b-versatile", api_key: str =None):
        """
        Initialize Groq LLM
        
        Args:
            model_name: Groq model name (qwen2-72b-instruct, llama3-70b-8192, etc.)
            api_key: Groq API key (or set GROQ_API_KEY environment variable)
        """
        self.model_name = model_name
        self.api_key = api_key or os.environ.get("GROQ_API_KEY")
        
        if not self.api_key:
            raise ValueError("Groq API key is required. Set GROQ_API_KEY environment variable or pass api_key parameter.")
        
        self.llm = ChatGroq(
            groq_api_key=self.api_key,
            model_name=self.model_name,
            temperature=0.1,
            max_tokens=1024
        )
        
        print(f"Initialized Groq LLM with model: {self.model_name}")

    def generate_response(self, query: str, context: str, max_length: int = 500) -> str:
        """
        Generate response using retrieved context
        
        Args:
            query: User question
            context: Retrieved document context
            max_length: Maximum response length
            
        Returns:
            Generated response string
        """
        
        # Create prompt template
        prompt_template = PromptTemplate(
            input_variables=["context", "question"],
            template="""You are a helpful AI assistant. Use the following context to answer the question accurately and concisely.

Context:
{context}

Question: {question}

Answer: Provide a clear and informative answer based on the context above. If the context doesn't contain enough information to answer the question, say so."""
        )
        
        # Format the prompt
        formatted_prompt = prompt_template.format(context=context, question=query)
        
        try:
            # Generate response
            messages = [HumanMessage(content=formatted_prompt)]
            response = self.llm.invoke(messages)
            return response.content
            
        except Exception as e:
            return f"Error generating response: {str(e)}"
        
    def generate_response_simple(self, query: str, context: str) -> str:
        """
        Simple response generation without complex prompting
        
        Args:
            query: User question
            context: Retrieved context
            
        Returns:
            Generated response
        """
        simple_prompt = f"""Based on this context: {context}

Question: {query}

Answer:"""
        
        try:
            messages = [HumanMessage(content=simple_prompt)]
            response = self.llm.invoke(messages)
            return response.content
        except Exception as e:
            return f"Error: {str(e)}"
    

In [21]:
# Initialize Groq LLM (you'll need to set GROQ_API_KEY environment variable)
try:
    groq_llm = GroqLLM(api_key=os.getenv("GROQ_API_KEY"))
    print("Groq LLM initialized successfully!")
except ValueError as e:
    print(f"Warning: {e}")
    print("Please set your GROQ_API_KEY environment variable to use the LLM.")
    groq_llm = None

Initialized Groq LLM with model: llama-3.3-70b-versatile
Groq LLM initialized successfully!


In [22]:
rag_retriever.retrieve("Cloud Computing")

Retrieving documents for query: 'Cloud Computing'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 13.08it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_f55792bc_82',
  'content': '© 2022, Amazon Web Services, Inc. or its affiliates. All rights reserved. 10\nSection 1 key \ntakeaways\n• Cloud architecture is the practice of \napplying cloud characteristics to a solution \nthat uses cloud services and features to \nmeet an organization’s technical needs \nand business use cases\n• You can use AWS services to create highly \navailable, scalable, and reliable \narchitectures\nSome key takeaways from this section of the module include:\n• Cloud architecture is the practice of applying cloud characteristics to a solution that uses cloud \nservices and features to meet an organization’s technical needs and business use cases\n• You can use AWS services to create highly available, scalable, and reliable architectures\nAWS Training and Certification Module 2: Introducing Cloud Architecting \n© 2023, Amazon Web Services, Inc. or its affiliates. All rights reserved. 13',
  'metadata': {'file_type': 'pdf',
   'source_file': 'MOD2.pdf

In [26]:
### Simple RAG pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

### Initialize the Groq LLM (set your GROQ_API_KEY in environment)
groq_api_key = os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.3-70b-versatile",temperature=0.1,max_tokens=1024)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query,retriever,llm,top_k=3):
    ## retriever the context
    results=retriever.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    
    ## generate the answwer using GROQ LLM
    prompt=f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response=llm.invoke([prompt.format(context=context,query=query)])
    return response.content

In [27]:
answer=rag_simple("What are availability zones?",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'What are availability zones?'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 30.22it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)


Availability Zones are isolated locations within an AWS Region, each comprising one or more data centers, designed as independent failure zones for fault isolation and high availability.
